In [ ]:
import openai
import json

client = openai.Client(base_url="http://127.0.0.1:30000/v1", api_key="None")

# загружаю файлы (с примерами и правильными ответами)
with open('files/RI разметка/3 итерация/items_1000.jsonl', 'r') as f:
    items_data = [json.loads(line) for line in f]
with open('files/RI разметка/3 итерация/data_vector_gt_1000.txt', 'r') as f:
    ground_truth = f.read().strip().splitlines()

results = []

for i, item in enumerate(items_data):
    # print(type(item[1]))
    # print(item[2])
    request = {
        "role": "user",
        "content": f"""
Ты помощник поиска в маркетплейсе.
Тебе необходимо определить, соответствует ли товар и его характеристики поисковому запросу.

Классы запросов:
relevant - товары, которые можно купить по указанному запросу
irrelevant - товары, которые было бы странно увидеть по указанному запросу

Товар:
{item[1]['товар']}

Запрос:
{item[2]['запрос']}

Определи релевантность запроса товару. Отвечай только "relevant" или "irrelevant". Не пиши объяснений и ничего больше.
"""
    }
    
    response = client.chat.completions.create(
        model="Qwen2-VL-72B-Instruct-AWQ ",
        messages=[
            {"role": "system", 
             "content": "You are a helpful AI assistant"},
            request
        ],
        temperature=0,
        max_tokens=3,
    )
    
    prediction = response.choices[0].message.content.strip().lower()
    
    # сравниваю предсказание с правильным ответом, исключая #N/A
    ground_truth_label = ground_truth[i].strip().lower()
    if ground_truth_label != '#n/a':
        is_correct = prediction == ground_truth_label
        results.append({
            "запрос": item[2]['запрос'],
            "товар": item[1]['товар'],
            "предсказание": prediction,
            "реальность": ground_truth_label,
            "is_correct": is_correct
        })

# Подсчет общей Accuracy
count_correct = sum(1 for item in results if item['is_correct'] == True)
total_predictions = len(results)
accuracy = count_correct / total_predictions if total_predictions > 0 else 0
print(f"Accuracy: {accuracy:.2%}")

# Accuracy для relevant
relevant_correct = sum(1 for item in results if item['реальность'] == 'relevant' and item['is_correct'])
relevant_total = sum(1 for item in results if item['реальность'] == 'relevant')
relevant_accuracy = relevant_correct / relevant_total if relevant_total > 0 else 0

# Accuracy для irrelevant
irrelevant_correct = sum(1 for item in results if item['реальность'] == 'irrelevant' and item['is_correct'])
irrelevant_total = sum(1 for item in results if item['реальность'] == 'irrelevant')
irrelevant_accuracy = irrelevant_correct / irrelevant_total if irrelevant_total > 0 else 0

print(f"Accuracy for relevant: {relevant_accuracy:.2%}")
print(f"Accuracy for irrelevant: {irrelevant_accuracy:.2%}")